# Section 508.3.3 - Separation
## Building Code
```
No separation is required between nonseparated occupancies. 

Exceptions: 

 1. Group H-2, H-3, H-4 and H-5 occupancies shall be separated from all other occupancies in accordance with Section 508.4. 

 2. Group I-1, R-1, R-2 and R-3 dwelling units and sleeping units  shall be separated from other dwelling or sleeping units and from other occupancies contiguous to them in accordance with the requirements of Section 420.
```



In [3]:
import pandas as pd
from IPython.display import display, HTML
from SPARQLWrapper import SPARQLWrapper, JSON

In [19]:

# Set up the SPARQL endpoint
endpoint_url = "http://localhost:3030/SmartReviewHotel/sparql"
sparql = SPARQLWrapper(endpoint_url)

with open("..\..\..\SPARQL\\uniqueOccupancy.sparql", "r") as file:
    sparql_query = file.read()
    
# Set up SPARQL query
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Process and format the results
story_dict = {}

for row in results["results"]["bindings"]:
    # Retrieve values from the SPARQL result row
    level_value = row["levelValue"]["value"]
    occupancy_value = row["uniqueOccupancies"]["value"]
    occupancies = occupancy_value.split(", ")
    
    if level_value not in story_dict:
        story_dict[level_value] = []
    
    story_dict[level_value].extend(occupancies)

data = []
# Format the data for the DataFrame
for level, occupancies in sorted(story_dict.items()):
    unique_occupancies = sorted(set(occupancies))  # Remove duplicates and sort for display
    occupancy_count = len(unique_occupancies)
    
    contains_exception1 = any(o in ["H-2", "H-3", "H-4", "H-5"] for o in unique_occupancies)
    contains_exception2 = any(o in ["I-1", "R-1", "R-2", "R-3"] for o in unique_occupancies)
    
    if occupancy_count == 1:
        exception1_result = "Passed"
        exception2_result = "Passed"
        reason = "The Separation Group only contains one occupancy. There can be no violation of Exception 1 or Exception 2."
    else:
        exception1_result = "Failed" if contains_exception1 else "Passed"
        exception2_result = "Passed" if contains_exception2 else "Failed"
        
        reason_parts = []
        if not contains_exception1:
            reason_parts.append("The Separation Group does not contain an occupancy described in Exception 1.")
        if not contains_exception2:
            reason_parts.append("The Separation Group does not contain an occupancy described in Exception 2.")
        
        reason = " ".join(reason_parts) if reason_parts else "The Separation Group does not contain the required occupancies for Exception 1 or Exception 2."
        
    data.append({
        "Separation Group": level,
        "Occupancies": ", ".join(unique_occupancies),
        "Exception 1 Result": exception1_result,
        "Exception 2 Result": exception2_result,
        "Reason": reason
    })

# Create DataFrame
df = pd.DataFrame(data)


In [20]:
# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame as a scrollable table
def render_scrollable_table(dataframe, max_height=300):
    display(HTML(f"""
    <div style="max-height: {max_height}px; overflow-y: scroll;">
        {dataframe.to_html(index=False)}
    </div>
    """))

render_scrollable_table(df)

Separation Group,Occupancies,Exception 1 Result,Exception 2 Result,Reason
01,"A-2, A-3, B, R-1",Passed,Passed,The Separation Group does not contain an occupancy described in Exception 1.
02,R-1,Passed,Passed,The Separation Group only contains one occupancy. There can be no violation of Exception 1 or Exception 2.
03,R-1,Passed,Passed,The Separation Group only contains one occupancy. There can be no violation of Exception 1 or Exception 2.
04,R-1,Passed,Passed,The Separation Group only contains one occupancy. There can be no violation of Exception 1 or Exception 2.
